In [1]:
from sentence_transformers import SentenceTransformer
import sqlite3
import hashlib
import json
from config import db_path


Database path: C:\Users\Admin\Desktop\Multimodal\database\database\multimodal_rag.db


In [2]:
model = SentenceTransformer('all-mpnet-base-v2')
texts = ["Artificial Intelligence is the simulation of human intelligence in machines", "The capital of France is Paris."] 

In [3]:
conn = sqlite3.connect(db_path)

In [4]:
for text in texts:
    embedding = model.encode(text)
    content_hash = hashlib.sha256(text.encode()).hexdigest()
    
    conn.execute('''
        INSERT INTO embeddings (modality, content_hash, embedding, metadata)
        VALUES (?, ?, ?, ?)
        ON CONFLICT(content_hash) DO UPDATE SET
            embedding = excluded.embedding,
            metadata = excluded.metadata,
            version = version + 1
    ''', ('text', content_hash, embedding.tobytes(), json.dumps({"content": text})))

In [5]:
conn.commit()
conn.close()
print("Text processing complete!")

Text processing complete!
